In [ ]:
import numpy as np
import polyscope as ps

In [ ]:
from chromalab.draw import IshiharaPlate, configurePolyscopeFor2D, configurePolyscopeForEvenOdd
from chromalab.observer import Observer, getSampledHyperCube
from chromalab.spectra import Illuminant
from chromalab.inks import get_metamer_buckets
from chromalab.maxdisplaybasis import TetraDisplayGamut

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

In [ ]:
ps.init()
configurePolyscopeForEvenOdd(target_sleep=80, vsync=False)
# im_h, im_w = 912, 1140
# configurePolyscopeFor2D(im_w, im_h)

In [ ]:
wavelengths = np.arange(400, 700, 5)
d65 = Illuminant.get("D65")

tetrachromat = Observer.tetrachromat(wavelengths=wavelengths, verbose=True) 
# maxbasis = MaxBasis(tetrachromat, verbose=True)
# viz = PSWrapper(tetrachromat, maxbasis, itemsToDisplay=PSWrapper.ItemsToDisplay.BOTH, displayBasis=DisplayBasisType.MAXBASIS, verbose=True) 

In [ ]:
# point_cloud = tetrachromat.get_optimal_point_cloud()

In [ ]:
hypercube = getSampledHyperCube(0.05, 4)

In [ ]:
factor = 10000
mdb = TetraDisplayGamut.loadTutenLabDisplay(tetrachromat, led_indices=[0, 3, 4, 5]) # R O C V
intensities = mdb.primary_intensities.T * 10000 # columns are the ratios, also 10000 is the factor in order to get the get buckets function to work

In [ ]:
all_lms_intensities = (intensities@hypercube.T).T # multiply all possible led combinations with the intensities
buckets = get_metamer_buckets(all_lms_intensities, axis=2)

In [ ]:
dist, (metamer_1, metamer_2) = buckets[0]

In [ ]:
weights_4tup = mdb.convertActivationsToIntensities(np.array([np.array(metamer_1)/factor, np.array(metamer_2)/factor]).T)
# insert 0 for G and B channels
weights = np.insert(arr=weights_4tup, obj=[1,1], values=0, axis=1)

In [ ]:
print((mdb.primary_intensities.T@weights_4tup.T).T) # checking if they are metamers again after being transformed back to weights!!

In [ ]:
ip = IshiharaPlate(weights[0], weights[1], 89, image_size=912)

In [ ]:
ip.generate_plate()

In [ ]:
while True:
    ps.full_frame_tick()